# Phase 1: Best Configuration Selection (Local, Google Colab & Kaggle)

This notebook automates the selection of the best model configuration from MLflow
based on metrics and benchmarking results, then performs final training and model conversion.


## Workflow

1. **Best Model Selection**: Query MLflow benchmark runs, join to training runs via grouping tags (`code.study_key_hash`, `code.trial_key_hash`), select best using normalized composite scoring
2. **Artifact Acquisition**: Download the best model's checkpoint using fallback strategy (local disk → drive restore → MLflow download)
3. **Final Training**: Optionally retrain with best config on full dataset (if not already final training)
4. **Model Conversion**: Convert the final model to ONNX format using canonical path structure


## Important

- This notebook **executes on Local, Google Colab, or Kaggle** (not on Azure ML compute)
- Requires MLflow tracking to be set up (Azure ML workspace or local SQLite)
- All computation happens on the platform's GPU (if available) or CPU
- **Storage & Persistence**:
  - **Local**: Outputs saved to `outputs/` directory in repository root
  - **Google Colab**: Checkpoints are automatically saved to Google Drive for persistence across sessions
  - **Kaggle**: Outputs in `/kaggle/working/` are automatically persisted - no manual backup needed
- The notebook must be **re-runnable end-to-end**
- Uses the dataset path specified in the data config (from `config/data/*.yaml`), typically pointing to a local folder included in the repository
- **Session Management**:
  - **Local**: No session limits, outputs persist in repository
  - **Colab**: Sessions timeout after 12-24 hours (depending on Colab plan). Checkpoints are saved to Drive automatically.
  - **Kaggle**: Sessions have time limits based on your plan. All outputs are automatically saved.


## Step 1: Environment Detection

The notebook automatically detects the execution environment (local, Google Colab, or Kaggle) and adapts its behavior accordingly.


In [1]:
import os
from pathlib import Path
# Detect execution environment
IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IS_LOCAL = not IN_COLAB and not IN_KAGGLE
# Set platform-specific constants
if IN_COLAB:
    PLATFORM = "colab"
    BASE_DIR = Path("/content")
    BACKUP_ENABLED = True
elif IN_KAGGLE:
    PLATFORM = "kaggle"
    BASE_DIR = Path("/kaggle/working")
    BACKUP_ENABLED = False
else:
    PLATFORM = "local"
    BASE_DIR = None
    BACKUP_ENABLED = False
print(f"✓ Detected environment: {PLATFORM.upper()}")
print(f"Platform: {PLATFORM}")
print(
    f"Base directory: {BASE_DIR if BASE_DIR else 'Current working directory'}")
print(f"Backup enabled: {BACKUP_ENABLED}")


✓ Detected environment: LOCAL
Platform: local
Base directory: Current working directory
Backup enabled: False


### Install Required Packages

Install required packages based on the execution environment.


In [2]:
# Install required packages
if IS_LOCAL:
    print("For local environment, please:")
    print("1. Create conda environment: conda env create -f config/environment/conda.yaml")
    print("2. Activate: conda activate resume-ner-training")
    print("3. Restart kernel after activation")
    print("\nIf you've already done this, you can continue to the next cell.")
    print("\nInstalling Azure ML SDK (required for imports)...")
    # Install Azure ML packages even for local (in case conda env not activated)
    %pip install "azure-ai-ml>=1.0.0" --quiet
    %pip install "azure-identity>=1.12.0" --quiet
    %pip install azureml-defaults --quiet
    %pip install azureml-mlflow --quiet
else:
    # Core ML libraries
    %pip install "transformers>=4.35.0,<5.0.0" --quiet
    %pip install "safetensors>=0.4.0" --quiet
    %pip install "datasets>=2.12.0" --quiet

    # ML utilities
    %pip install "numpy>=1.24.0,<2.0.0" --quiet
    %pip install "pandas>=2.0.0" --quiet
    %pip install "scikit-learn>=1.3.0" --quiet

    # Utilities
    %pip install "pyyaml>=6.0" --quiet
    %pip install "tqdm>=4.65.0" --quiet
    %pip install "seqeval>=1.2.2" --quiet
    %pip install "sentencepiece>=0.1.99" --quiet

    # Experiment tracking
    %pip install mlflow --quiet
    %pip install optuna --quiet

    # Azure ML SDK (required for orchestration imports)
    %pip install "azure-ai-ml>=1.0.0" --quiet
    %pip install "azure-identity>=1.12.0" --quiet
    %pip install azureml-defaults --quiet
    %pip install azureml-mlflow --quiet

    # ONNX support
    %pip install onnxruntime --quiet
    %pip install "onnx>=1.16.0" --quiet
    %pip install "onnxscript>=0.1.0" --quiet

    print("✓ All dependencies installed")


For local environment, please:
1. Create conda environment: conda env create -f config/environment/conda.yaml
2. Activate: conda activate resume-ner-training
3. Restart kernel after activation

If you've already done this, you can continue to the next cell.

Installing Azure ML SDK (required for imports)...
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Step 2: Repository Setup

**Note**: Repository setup is only needed for Colab/Kaggle environments. Local environments should already have the repository cloned.


In [3]:
# Repository setup - only needed for Colab/Kaggle
if not IS_LOCAL:
    if IN_KAGGLE:
        # For Kaggle
        !git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /kaggle/working/resume-ner-azureml
    elif IN_COLAB:
        # For Google Colab
        !git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /content/resume-ner-azureml
else:
    print("✓ Local environment detected - assuming repository already exists")

# Set up paths
if not IS_LOCAL:
    ROOT_DIR = BASE_DIR / "resume-ner-azureml"
else:
    # For local, try to find repo root
    ROOT_DIR = Path.cwd()
    # Try to find the repo root by looking for config directory
    while ROOT_DIR.parent != ROOT_DIR:
        if (ROOT_DIR / "config").exists() and (ROOT_DIR / "src").exists():
            break
        ROOT_DIR = ROOT_DIR.parent

CONFIG_DIR = ROOT_DIR / "config"
SRC_DIR = ROOT_DIR / "src"

# Add src to path
import sys
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print(f"✓ Repository root: {ROOT_DIR}")
print(f"✓ Config directory: {CONFIG_DIR}")
print(f"✓ Source directory: {SRC_DIR}")

# Verify repository structure
required_dirs = [CONFIG_DIR, SRC_DIR]
for dir_path in required_dirs:
    if not dir_path.exists():
        raise ValueError(f"Required directory not found: {dir_path}")
print("✓ Repository structure verified")


✓ Local environment detected - assuming repository already exists
✓ Repository root: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml
✓ Config directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\config
✓ Source directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\src
✓ Repository structure verified


## Step 3: Load Configuration

Load experiment configuration and define experiment naming convention.


In [4]:
from orchestration.config_loader import load_experiment_config
from orchestration import EXPERIMENT_NAME
from shared.yaml_utils import load_yaml

# Load experiment config
experiment_config = load_experiment_config(CONFIG_DIR, EXPERIMENT_NAME)
print(f"✓ Loaded experiment config: {experiment_config.name}")

# Load best model selection configs
tags_config = load_yaml(CONFIG_DIR / "tags.yaml")
selection_config = load_yaml(CONFIG_DIR / "best_model_selection.yaml")
conversion_config = load_yaml(CONFIG_DIR / "conversion.yaml")
acquisition_config = load_yaml(CONFIG_DIR / "artifact_acquisition.yaml")

print(f"✓ Loaded tags config")
print(f"✓ Loaded best model selection config")
print(f"✓ Loaded conversion config")
print(f"✓ Loaded artifact acquisition config")

# Define experiment names (discovery happens after MLflow setup in Cell 4)
experiment_name = experiment_config.name
benchmark_experiment_name = f"{experiment_name}-benchmark"
training_experiment_name = f"{experiment_name}-training"  # For final training runs
conversion_experiment_name = f"{experiment_name}-conversion"

print(f"\n✓ Experiment names defined:")
print(f"  - Benchmark: {benchmark_experiment_name}")
print(f"  - Training: {training_experiment_name}")
print(f"  - Conversion: {conversion_experiment_name}")
print(f"\nNote: Experiment discovery will happen after MLflow setup (Cell 4)")

c:\Users\HOANG PHI LONG DANG\Miniconda3\envs\resume-ner-training\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Loaded experiment config: resume_ner_baseline
✓ Loaded tags config
✓ Loaded best model selection config
✓ Loaded conversion config
✓ Loaded artifact acquisition config

✓ Experiment names defined:
  - Benchmark: resume_ner_baseline-benchmark
  - Training: resume_ner_baseline-training
  - Conversion: resume_ner_baseline-conversion

Note: Experiment discovery will happen after MLflow setup (Cell 4)


## Step 4: Setup MLflow

Setup MLflow tracking with fallback to local if Azure ML is unavailable.


In [5]:
from shared.mlflow_setup import setup_mlflow_from_config
import mlflow

# Setup MLflow tracking (use training experiment for setup - actual queries use discovered experiments)
tracking_uri = setup_mlflow_from_config(
    experiment_name=training_experiment_name,
    config_dir=CONFIG_DIR,
    fallback_to_local=True,
)

print(f"✓ MLflow tracking URI: {tracking_uri}")
print(f"✓ MLflow experiment: {training_experiment_name}")

# Discover HPO and benchmark experiments from MLflow (after setup)
from mlflow.tracking import MlflowClient

client = MlflowClient()
all_experiments = client.search_experiments()

# Find HPO experiments (format: {experiment_name}-hpo-{backbone})
hpo_experiments = {}
for exp in all_experiments:
    if exp.name.startswith(f"{experiment_name}-hpo-"):
        backbone = exp.name.replace(f"{experiment_name}-hpo-", "")
        hpo_experiments[backbone] = {
            "name": exp.name,
            "id": exp.experiment_id
        }

# Find benchmark experiment
benchmark_experiment = None
for exp in all_experiments:
    if exp.name == benchmark_experiment_name:
        benchmark_experiment = {
            "name": exp.name,
            "id": exp.experiment_id
        }
        break

print(f"\n✓ Found {len(hpo_experiments)} HPO experiment(s):")
for backbone, exp_info in hpo_experiments.items():
    print(f"  - {exp_info['name']} (backbone: {backbone})")

if benchmark_experiment:
    print(f"✓ Benchmark experiment: {benchmark_experiment['name']}")
else:
    print(f"⚠ Benchmark experiment not found: {benchmark_experiment_name}")

print(f"✓ Training experiment: {training_experiment_name} (for final training runs)")
print(f"✓ Conversion experiment: {conversion_experiment_name}")


2026-01-06 00:04:47,011 - shared.mlflow_setup - INFO - Azure ML enabled in config, attempting to connect...
2026-01-06 00:04:50,264 - shared.mlflow_setup - WARNING - [DEBUG] Initial env check - subscription_id: False, resource_group: False, client_id: False, client_secret: False, tenant_id: False
2026-01-06 00:04:50,265 - shared.mlflow_setup - INFO - Attempting to load credentials from config.env at: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\config.env
2026-01-06 00:04:50,266 - shared.mlflow_setup - INFO - Loading credentials from c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\config.env
2026-01-06 00:04:50,267 - shared.mlflow_setup - INFO - Loaded subscription/resource group from config.env
2026-01-06 00:04:50,269 - shared.mlflow_setup - INFO - Loaded service principal credentials from config.env
2026-01-06 00:04:50,270 - shared.mlflow_setup - WARNING - [DEBUG] Platform detected: local
2026-01-06 00:04:50,271 - shared.mlflow_setup - WARNING - [DEBUG] Service Princip

✓ MLflow tracking URI: azureml://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws
✓ MLflow experiment: resume_ner_baseline-training

✓ Found 2 HPO experiment(s):
  - resume_ner_baseline-hpo-distilbert (backbone: distilbert)
  - resume_ner_baseline-hpo-distilroberta (backbone: distilroberta)
✓ Benchmark experiment: resume_ner_baseline-benchmark
✓ Training experiment: resume_ner_baseline-training (for final training runs)
✓ Conversion experiment: resume_ner_baseline-conversion


## Step 5: Drive Backup Setup (Colab Only)

Setup Google Drive backup/restore for Colab environments.


In [6]:
from pathlib import Path

# Fix numpy/pandas compatibility before importing orchestration modules
try:
    from orchestration.drive_backup import create_colab_store
except (ValueError, ImportError) as e:
    if "numpy.dtype size changed" in str(e) or "numpy" in str(e).lower():
        print("⚠ Numpy/pandas compatibility issue detected. Fixing...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "--no-cache-dir", "numpy>=1.24.0,<2.0.0", "pandas>=2.0.0", "--quiet"])
        print("✓ Numpy/pandas reinstalled. Please restart the kernel and re-run this cell.")
        raise RuntimeError("Please restart kernel after numpy/pandas fix")
    else:
        raise

# Mount Google Drive and create backup store (Colab only - Kaggle doesn't need this)
DRIVE_BACKUP_DIR = None
drive_store = None
restore_from_drive = None

if IN_COLAB:
    drive_store = create_colab_store(ROOT_DIR, CONFIG_DIR)
    if drive_store:
        BACKUP_ENABLED = True
        DRIVE_BACKUP_DIR = drive_store.backup_root
        # Create restore function wrapper
        def restore_from_drive(local_path: Path, is_directory: bool = False) -> bool:
            """Restore file/directory from Drive backup."""
            try:
                expect = "dir" if is_directory else "file"
                result = drive_store.restore(local_path, expect=expect)
                return result.ok
            except Exception as e:
                print(f"⚠ Drive restore failed: {e}")
                return False
        print(f"✓ Google Drive mounted")
        print(f"✓ Backup base directory: {DRIVE_BACKUP_DIR}")
        print(f"\nNote: All outputs/ will be mirrored to: {DRIVE_BACKUP_DIR / 'outputs'}")
    else:
        BACKUP_ENABLED = False
        print("⚠ Warning: Could not mount Google Drive. Backup to Google Drive will be disabled.")
elif IN_KAGGLE:
    print("✓ Kaggle environment detected - outputs are automatically persisted (no Drive mount needed)")
    BACKUP_ENABLED = False
else:
    # Local environment
    print("✓ Local environment detected - outputs will be saved to repository (no Drive backup needed)")
    BACKUP_ENABLED = False


✓ Local environment detected - outputs will be saved to repository (no Drive backup needed)


## Step 6: Best Model Selection

Query MLflow benchmark runs, join to training runs via grouping tags, and select the best model using normalized composite scoring.


In [7]:
from orchestration.jobs.selection.mlflow_selection import find_best_model_from_mlflow
from orchestration.jobs.selection.artifact_acquisition import acquire_best_model_checkpoint
from pathlib import Path
from typing import Optional, Callable, Dict, Any

# Validate experiments
if benchmark_experiment is None:
    raise ValueError(f"Benchmark experiment '{benchmark_experiment_name}' not found. Run benchmark jobs first.")

if not hpo_experiments:
    raise ValueError(f"No HPO experiments found. Run HPO jobs first.")

# Find best model
best_model = find_best_model_from_mlflow(
    benchmark_experiment=benchmark_experiment,
    hpo_experiments=hpo_experiments,
    tags_config=tags_config,
    selection_config=selection_config,
    use_python_filtering=True,
)

if best_model is None:
    raise ValueError("Could not find best model from MLflow.")

# Extract lineage information from best_model for final training tags
from orchestration.jobs.final_training import extract_lineage_from_best_model
lineage = extract_lineage_from_best_model(best_model)
print(f"✓ Extracted lineage information: {list(lineage.keys())}")

# Acquire checkpoint
best_checkpoint_dir = acquire_best_model_checkpoint(
    best_run_info=best_model,
    root_dir=ROOT_DIR,
    config_dir=CONFIG_DIR,
    acquisition_config=acquisition_config,
    selection_config=selection_config,
    platform=PLATFORM,
    restore_from_drive=restore_from_drive if "restore_from_drive" in locals() else None,
    drive_store=drive_store if "drive_store" in locals() else None,
    in_colab=IN_COLAB,
)

print(f"\n✓ Best model checkpoint available at: {best_checkpoint_dir}")

2026-01-06 00:04:55,179 - orchestration.jobs.selection.mlflow_selection - INFO - Finding best model from MLflow
2026-01-06 00:04:55,181 - orchestration.jobs.selection.mlflow_selection - INFO -   Benchmark experiment: resume_ner_baseline-benchmark
2026-01-06 00:04:55,181 - orchestration.jobs.selection.mlflow_selection - INFO -   HPO experiments: 2
2026-01-06 00:04:55,182 - orchestration.jobs.selection.mlflow_selection - INFO -   Objective metric: macro-f1
2026-01-06 00:04:55,182 - orchestration.jobs.selection.mlflow_selection - INFO -   Composite weights: F1=0.70, Latency=0.30
2026-01-06 00:04:55,183 - orchestration.jobs.selection.mlflow_selection - INFO - Querying benchmark runs...


🔍 Finding best model from MLflow...
   Benchmark experiment: resume_ner_baseline-benchmark
   HPO experiments: 2
   Objective metric: macro-f1
   Composite weights: F1=0.70, Latency=0.30

📊 Querying benchmark runs...


2026-01-06 00:04:55,331 - orchestration.jobs.selection.mlflow_selection - INFO - Found 2 finished benchmark runs
2026-01-06 00:04:55,331 - orchestration.jobs.selection.mlflow_selection - INFO - Found 2 benchmark runs with required metrics and grouping tags
2026-01-06 00:04:55,332 - orchestration.jobs.selection.mlflow_selection - INFO - Preloading trial and refit runs from HPO experiments...


   Found 2 benchmark runs with required metrics and grouping tags

🔗 Preloading trial runs (metrics) and refit runs (artifacts) from HPO experiments...
   resume_ner_baseline-hpo-distilbert: 10 finished runs, 4 trial runs, 2 refit runs


2026-01-06 00:04:55,619 - orchestration.jobs.selection.mlflow_selection - INFO - Built trial lookup with 2 unique (study_hash, trial_hash) pairs
2026-01-06 00:04:55,619 - orchestration.jobs.selection.mlflow_selection - INFO - Built refit lookup with 2 unique (study_hash, trial_hash) pairs
2026-01-06 00:04:55,620 - orchestration.jobs.selection.mlflow_selection - INFO - Joining benchmark runs with trial runs and refit runs...
2026-01-06 00:04:55,621 - orchestration.jobs.selection.mlflow_selection - INFO - Found 2 candidate(s) with both benchmark and training metrics
2026-01-06 00:04:55,622 - orchestration.jobs.selection.mlflow_selection - INFO - Computing composite scores...
2026-01-06 00:04:55,623 - orchestration.jobs.selection.mlflow_selection - INFO - Best model selected:
2026-01-06 00:04:55,623 - orchestration.jobs.selection.mlflow_selection - INFO -   Artifact Run ID: 63f2e01b-ccc3-4b91-b708-0c5907ed819b
2026-01-06 00:04:55,624 - orchestration.jobs.selection.mlflow_selection - INFO 

   resume_ner_baseline-hpo-distilroberta: 0 finished runs, 0 trial runs, 0 refit runs
   Built trial lookup with 2 unique (study_hash, trial_hash) pairs
   Built refit lookup with 2 unique (study_hash, trial_hash) pairs

🔗 Joining benchmark runs with trial runs (metrics) and refit runs (artifacts)...
   Found 2 candidate(s) with both benchmark and training metrics

✅ Best model selected:
   Run ID: 63f2e01b-ccc3-4b91-b708-0c5907ed819b
   Experiment: resume_ner_baseline-hpo-distilbert
   Backbone: distilbert
   F1 Score: 0.4316
   Latency: 6.00 ms
   Composite Score: 0.7000
✓ Extracted lineage information: ['hpo_study_key_hash', 'hpo_trial_key_hash', 'hpo_trial_run_id', 'hpo_refit_run_id']

✓ Best model checkpoint available at: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\outputs\best_model_selection\local\distilbert\sel_350a79aa_f349f4af


In [8]:
# Check if selected run is already final training (skip retraining if so)
stage_tag = tags_config["process"]["stage"]
trained_on_full_data_tag = tags_config["training"]["trained_on_full_data"]

is_final_training = best_model["tags"].get(stage_tag) == "final_training"
used_full_data = (
    best_model["tags"].get(trained_on_full_data_tag) == "true" or
    best_model["params"].get("use_combined_data", "false").lower() == "true"
)

SKIP_FINAL_TRAINING = is_final_training and used_full_data

if SKIP_FINAL_TRAINING:
    final_checkpoint_dir = best_checkpoint_dir


## Step 7: Final Training

Run final training with best configuration if needed.


In [9]:
if not SKIP_FINAL_TRAINING:
    print("🔄 Starting final training with best configuration...")
    from orchestration.jobs.final_training import execute_final_training
    # Execute final training (uses final_training.yaml via load_final_training_config)
    # Will automatically reuse existing complete runs if run.mode: reuse_if_exists in final_training.yaml
    final_checkpoint_dir = execute_final_training(
        root_dir=ROOT_DIR,
        config_dir=CONFIG_DIR,
        best_model=best_model,
        experiment_config=experiment_config,
        lineage=lineage,
        training_experiment_name=training_experiment_name,
        platform=PLATFORM,
    )
else:
    print("✓ Skipping final training - using selected checkpoint")

# Backup final checkpoint to Google Drive if in Colab
if IN_COLAB and drive_store and final_checkpoint_dir:
    try:
        # Ensure checkpoint path is absolute and resolved
        from pathlib import Path
        checkpoint_path = Path(final_checkpoint_dir).resolve()
        
        print(f"\n📦 Backing up final training checkpoint to Google Drive...")
        print(f"  Local path: {checkpoint_path}")
        print(f"  Root dir: {drive_store.root_dir}")
        print(f"  Backup root: {drive_store.backup_root}")
        
        result = drive_store.backup(checkpoint_path, expect="dir")
        if result.ok:
            print(f"✓ Successfully backed up final checkpoint to Google Drive")
            print(f"  Drive path: {result.dst}")
        else:
            print(f"⚠ Drive backup failed, but checkpoint is available locally")
            print(f"  Reason: {result.reason}")
            if result.error:
                print(f"  Error: {result.error}")
            print(f"  Attempted drive path: {result.dst}")
    except Exception as e:
        print(f"⚠ Drive backup error: {e}")
        print(f"  Checkpoint is still available locally at: {final_checkpoint_dir}")
        import traceback
        traceback.print_exc()


🔄 Starting final training with best configuration...
✓ Final training config loaded from final_training.yaml
✓ Output directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\outputs\final_training\local\distilbert\spec_81710c3324325ad0_exec_e2ae9d18a17895d2\v1
✓ Found existing completed final training run
  Output directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\outputs\final_training\local\distilbert\spec_81710c3324325ad0_exec_e2ae9d18a17895d2\v1
  Checkpoint: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\outputs\final_training\local\distilbert\spec_81710c3324325ad0_exec_e2ae9d18a17895d2\v1\checkpoint
  Reusing existing checkpoint (run.mode: reuse_if_exists)


## Step 8: Model Conversion & Optimization

Convert the final trained model to ONNX format with optimization.

In [10]:
# Extract parent training information for conversion
from orchestration.metadata_manager import load_metadata
from pathlib import Path

# Load metadata from final training output directory
final_training_metadata_path = final_checkpoint_dir.parent / "metadata.json"
parent_spec_fp = None
parent_exec_fp = None
parent_training_run_id = None

if final_training_metadata_path.exists():
    try:
        metadata = load_metadata(final_training_metadata_path)
        parent_spec_fp = metadata.get("spec_fp")
        parent_exec_fp = metadata.get("exec_fp")
        parent_training_run_id = metadata.get("mlflow", {}).get("run_id")
    except Exception as e:
        print(f"⚠ Warning: Could not load metadata: {e}")

# Fallback: try to extract from path if metadata missing
if not parent_spec_fp or not parent_exec_fp:
    path_parts = final_checkpoint_dir.parts
    try:
        spec_idx = next(i for i, p in enumerate(path_parts) if p.startswith("spec_"))
        spec_exec_part = path_parts[spec_idx]
        # Extract spec_fp and exec_fp from "spec_{spec_fp}_exec_{exec_fp}"
        parts = spec_exec_part.split("_")
        if len(parts) >= 4:
            parent_spec_fp = parts[1]  # spec_fp
            parent_exec_fp = parts[3]  # exec_fp
        else:
            raise ValueError("Could not extract fingerprints from path")
    except (StopIteration, ValueError, IndexError) as e:
        raise ValueError(
            f"Could not extract parent training fingerprints from path: {final_checkpoint_dir}\n"
            "Please ensure final training completed successfully and metadata.json exists."
        ) from e

print(f"✓ Parent training fingerprints:")
print(f"  - spec_fp: {parent_spec_fp}")
print(f"  - exec_fp: {parent_exec_fp}")
if parent_training_run_id:
    print(f"  - MLflow run ID: {parent_training_run_id[:12]}...")
else:
    print(f"  ⚠ Warning: Parent training run ID not found - lineage will be partial")

# Get parent training output directory (checkpoint parent)
parent_training_output_dir = final_checkpoint_dir.parent

print(f"\n🔄 Starting model conversion...")
from orchestration.jobs.conversion import execute_conversion

# Execute conversion (uses conversion.yaml via load_conversion_config)
conversion_output_dir = execute_conversion(
    root_dir=ROOT_DIR,
    config_dir=CONFIG_DIR,
    parent_training_output_dir=parent_training_output_dir,
    parent_spec_fp=parent_spec_fp,
    parent_exec_fp=parent_exec_fp,
    experiment_config=experiment_config,
    conversion_experiment_name=conversion_experiment_name,
    platform=PLATFORM,
    parent_training_run_id=parent_training_run_id,  # May be None, that's OK
)

# Find ONNX model file
onnx_files = list(conversion_output_dir.glob("*.onnx"))
if onnx_files:
    onnx_model_path = onnx_files[0]
    print(f"\n✓ Conversion completed successfully!")
    print(f"  ONNX model: {onnx_model_path}")
    print(f"  Model size: {onnx_model_path.stat().st_size / (1024 * 1024):.2f} MB")
else:
    print(f"\n⚠ Warning: No ONNX model file found in {conversion_output_dir}")

# Backup conversion output to Google Drive if in Colab
if IN_COLAB and drive_store and conversion_output_dir:
    try:
        # Ensure output path is absolute and resolved
        output_path = Path(conversion_output_dir).resolve()
        
        print(f"\n📦 Backing up conversion output to Google Drive...")
        print(f"  Local path: {output_path}")
        
        result = drive_store.backup(output_path, expect="dir")
        if result.ok:
            print(f"✓ Successfully backed up conversion output to Google Drive")
            print(f"  Drive path: {result.dst}")
        else:
            print(f"⚠ Drive backup failed, but output is available locally")
            print(f"  Reason: {result.reason}")
            if result.error:
                print(f"  Error: {result.error}")
    except Exception as e:
        print(f"⚠ Drive backup error: {e}")
        print(f"  Output is still available locally at: {conversion_output_dir}")
        import traceback
        traceback.print_exc()


ImportError: cannot import name 'load_metadata' from 'orchestration.metadata_manager' (c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\src\orchestration\metadata_manager.py)